# Cuaderno 34: Generación de columnas
## Otro modelo para el problema de corte de material (Cutting Stock Problem)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\S}{\mathcal{S}}$


Dados: 
* un conjunto de rollos, cada uno de longitud $L$; y,
* $m$ órdenes de corte, cada una de las cuales consiste en cortar de los rollos disponibles $b_i$ items de longitud $w_i$, con $w_i < L$, $i \in \{1, \ldots, m\}$.

El problema del corte de material (*Cutting Stock Problem*) consiste en determinar la manera de satisfacer todas las órdenes de corte empleando la menor cantidad posible de rollos. Consideraremos a continuación una formulación para este problema distinta a la formulación del Cuaderno 33.

Definimos un *patrón de corte* como una manera posible de cortar uno o más items de la longitud requerida por las órdenes a partir de *un* rollo de longitud $L$. Notar que un patrón de corte puede ser representado como un vector $s$ de $m$ componentes enteras que además satisface $\sum_{i=1}^m w_i s_i \leq L$. Es decir, el conjunto de todos los posibles patrones de corte está dado por:
$$
\S :=  \left\{ s \in \ZZ^m_+ \; : \; \sum_{i=1}^m w_i s_i \leq L \right\}.
$$

Definiendo una variable entera $x_s$ que indique el número de veces que se usa cada patrón de corte $s \in S$ en la solución, el problema de corte de material puede formularse como el siguiente programa lineal entero:
\begin{align*}
\min &\sum_{s \in \S} x_s\\ 
& \mbox{s.r.}\\
&\sum_{s \in \S} s_i x_s  \geq b_i, \quad \forall i \in \{1, \ldots, m\},\\
& x_s \in \ZZ, \quad \forall s \in \S.
\end{align*}

Llamaremos *problema master (PM)* al modelo anterior. Generalmente, este programa lineal entero tiene un número astronómico de variables, pues la cantidad de posibles patrones de corte crece exponencialmente como función de $m$. Problemas de este tipo pueden resolverse empleando una técnica conocida como **generación de columnas**.

Definimos al *problema master reducido (PMR)* como el modelo anterior, pero restringido a utilizar únicamente un subconjunto $\S' \subset \S$ de patrones de corte disponibles. Inicialmente, $\S'$ puede contener, por ejemplo, solamente *patrones homogéneos* que cortan un rollo en la mayor cantidad posible de items de la misma longitud (notar que esta cantidad es igual a $\lfloor \frac{L}{w_i} \rfloor$, para $i \in \{1, \ldots, m\}$). Llamando $s^1, \ldots, s^m$ a estos patrones homogéneos, inicialmente PMR tiene la forma:
\begin{align*}
\min &\sum_{i = 1}^m x_{s^i}\\ 
& \mbox{s.r.}\\
& \lfloor \frac{L}{w_i} \rfloor x_{s^i}  \geq b_i, \quad \forall i \in \{1, \ldots, m\},\\
& x_{s^i} \in \ZZ, \quad \forall i \in \{1, \ldots, m\}.
\end{align*}


El algoritmo de generación de columnas empieza por resolver la relajación lineal de PMR. Luego se resuelve el siguiente problema de costeo (PRICING): Determinar si existe algún patrón en $\S \setminus \S'$ tal que el costo reducido de la variable correspondiente en la relajación lineal de PM, respecto a los precios sombra dados por la solución del problema dual de PMR, sea estrictamente negativo. De ser así, se añade este nuevo patrón como una variable adicional a PMR y se repite el proceso. Caso contrario, la solución óptima de la relajación lineal de PMR es también una solución óptima de la relajación lineal de PM y la fase de generación de columnas termina. Finalmente, el modelo PMR se resuelve como un problema lineal entero. 

El algoritmo de generación de columnas permite encontrar la solución óptima de la relajación lineal de PM, aunque no necesariamente la solución óptima entera. Para resolver PM hasta la optimalidad, sería necesario aplicar una nueva fase de generación de columnas en cada nodo del árbol de branch-and-bound. Este nuevo algoritmo se conoce como *Branch-and-Price* y su uso está limitado por su gran complejidad computacional. 

Para el caso del problema del corte de material, sin embargo, el algoritmo (simple) de generación de columna permite obtener soluciones con brechas de optimalidad muy pequeñas.

Implementaremos a continuación el algoritmo de generación de columnas para resolver PM utilizando la interfaz Python del solver Gurobi.


Definimos primero los datos. Generaremos $m$ órdenes de corte, consistentes en cortar items de longitud $1, 3, \ldots, 2m+1$. La cantidad de items a cortar de cada tipo estará dada por un número aleatorio entre 1 y 50, obtenido llamando a la función `randint` del módulo `random`. La longitud de los rollos será $L := 3m$.

In [1]:
from gurobipy import *
import random as rm

# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de ordenes
m = 50

# Conjunto de indices para las ordenes, I:={1,...,m}
I = range(1, m+1)

# Diccionarios con las longitudes y cantidades demandadas de cada item
w = {} # longitudes 1, 3, ... , 2m+1
b = {} # demandas (aleatorio entre 1 y 50)
for i in I:
    w[i]= rm.randint(m, 4*m)
    b[i]= rm.randint(1, 50)

L = 5*m

# Este valor se utiliza para indicar el mayor indice de una variable en el modelo
K = m

print w
print b
print L

{1: 177, 2: 113, 3: 127, 4: 168, 5: 121, 6: 187, 7: 92, 8: 143, 9: 187, 10: 172, 11: 96, 12: 185, 13: 121, 14: 115, 15: 187, 16: 122, 17: 89, 18: 132, 19: 158, 20: 174, 21: 50, 22: 181, 23: 99, 24: 78, 25: 86, 26: 171, 27: 62, 28: 126, 29: 66, 30: 156, 31: 172, 32: 195, 33: 138, 34: 140, 35: 136, 36: 78, 37: 142, 38: 121, 39: 164, 40: 189, 41: 185, 42: 131, 43: 156, 44: 172, 45: 185, 46: 193, 47: 118, 48: 200, 49: 169, 50: 142}
{1: 38, 2: 13, 3: 21, 4: 16, 5: 30, 6: 26, 7: 38, 8: 13, 9: 50, 10: 46, 11: 37, 12: 35, 13: 6, 14: 31, 15: 49, 16: 44, 17: 41, 18: 1, 19: 20, 20: 34, 21: 25, 22: 13, 23: 44, 24: 29, 25: 49, 26: 23, 27: 17, 28: 47, 29: 28, 30: 28, 31: 28, 32: 31, 33: 23, 34: 20, 35: 15, 36: 10, 37: 33, 38: 5, 39: 44, 40: 43, 41: 47, 42: 20, 43: 14, 44: 43, 45: 30, 46: 29, 47: 34, 48: 46, 49: 5, 50: 25}
250


Definimos ahora el modelo reducido inicial PMR, sus variables y su función objetivo. Este modelo contiene únicamente patrones homogéneos que cortan rollos en items de la misma longitud.


In [42]:
# Crear el objeto modelo
pm = Model('cutting-stock')

# Crear las variables asociadas a patrones homogeneos
x = pm.addVars(I, name="x", vtype=GRB.INTEGER)

# Crear la funcion objetivo
pm.setObjective(x.sum('*'), GRB.MINIMIZE)


Añadimos las restricciones de satisfacción de la demanda. 

In [43]:
# Restricciones de demanda
r = pm.addConstrs((int(L/w[i])*x[i] >= b[i] for i in I), "demanda");


Resolvemos ahora la relajación lineal del modelo inicial. La relajación lineal de un modelo de programación lineal entera puede construirse llamando al método `relax` de la clase `Model`.

In [44]:
# Llamamos primero a update para actualizar los cambios
pm.update()

# Construimos la relajacion lineal de m
relax = pm.relax()

# Resolvemos la relacion lineal de m
relax.optimize()

Optimize a model with 20 rows, 20 columns and 20 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 5e+01]
Presolve removed 20 rows and 20 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4851667e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds
Optimal objective  3.485166667e+02


### Resolviendo el problema de costeo (PRICING)

El problema de costeo consiste en encontrar, o determinar que no existe, un nuevo patrón de corte $s \in \S$, tal que la variable correspondiente en la relajación lineal de PM tenga costo reducido negativo, con respecto a los valores $\pi_1, \ldots, \pi_m$ de las variables duales de la solución óptima de la relajación lineal de PMR encontrada en el paso anterior.

Suponer que este patrón de corte tiene la forma $s:= (y_1, \ldots, y_m)$, con $y_i$ enteros no negativos tales que $\sum_{i=1}^m w_i y_i \leq L$. El costo reducido de la variable $x_s$ asociada a este patrón es:
$$
\tilde{c}_s := c_s - \sum_{i = 1}^m \pi_i y_i,
$$
donde $c_s = 1$ es el coeficiente de $x_s$ en la función objetivo de PM. Observar que $\tilde{c}_s < 0$ si y solamente si $\sum_{i = 1}^m \pi_i y_i > 1$.

Por lo tanto, el problema de costeo puede reducirse a la solución del siguiente problema de la mochila (*knapsack problem*):
\begin{align*}
\max &\sum_{i = 1}^m \pi_i y_i\\ 
& \mbox{s.r.}\\
&\sum_{i=1}^m w_i y_i \leq L,\\
& y_i \in \ZZ_+, \quad \forall i \in \{1, \ldots, m\}.
\end{align*}

Si la solución óptima de este problema tiene un valor mayor a uno, la misma está asociada a un nuevo patrón de costo reducido negativo. Caso contrario, se demuestra que no existen más patrones con costo reducido negativo y que por tanto la solución óptima de la relajación lineal de PMR es también solución óptima de la relajación lineal de PM.

Para resolver este problema de la mochila, empezamos por recuperar los valores de $\pi_1, \ldots, \pi_m$ asociados a la solución dual del modelo `relax`. Esto se consigue accediendo al atributo `Pi` de cada restricción lineal. A su vez, para acceder a las restricciones lineales del modelo, se emplea el método `getConstrs()` de la clase `Model`.


In [45]:
# Recuperamos la solucion dual del problema master
pi = {}
i = 1
for c in relax.getConstrs():
    pi[i] = c.Pi
    i+= 1


Construimos ahora el modelo del problema de la mochila, empleando los valores de `pi`, `w` y `L`:

In [46]:
# Crear un objeto modelo auxiliar para el problema de la mochila
h = Model('knapsack')

# Crear las variables del problema de la mochila
y = h.addVars(I, name="y", vtype=GRB.INTEGER)

# Crear la funcion objetivo
h.setObjective(y.prod(pi, '*'), GRB.MAXIMIZE)

# Agregar la restriccion 
h.addConstr(y.prod(w, '*') <= L, 'capacidad')

h.update()

Resolvemos el problema de la mochila y verificamos si la solución óptima tiene un valor mayor a 1.

In [47]:
# Resolver modelo de pricing (problema de la mochila)
h.optimize()


Optimize a model with 1 rows, 20 columns and 20 nonzeros
Variable types: 0 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e+00, 4e+01]
  Objective range  [5e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 6e+01]
Found heuristic solution: objective 1.0000000
Presolve removed 0 rows and 11 columns
Presolve time: 0.00s
Presolved: 1 rows, 9 columns, 9 nonzeros
Variable types: 0 continuous, 9 integer (1 binary)

Root relaxation: objective 1.690476e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.69048    0    1    1.00000    1.69048  69.0%     -    0s
H    0     0                       1.6333333    1.69048  3.50%     -    0s
     0     0 infeasible    0         1.63333    1.63333  0.00%     -    0s

Explored 1 nodes (2 simplex iterations) in 0.15 seconds
Thread count was 4 (of 4 avail

Como el valor de la función objetivo es mayor a 1, agregamos una nueva variable al modelo PM, asociada al nuevo patrón encontrado. 

Para ello, construimos en primer lugar un objeto de tipo `Column` que almacenará la columna correspondiente a esta nueva variable. Los coeficientes (no nulos) de la columna se definen llamando al método `addTerms`. Finalmente, se llama a `addVar` para agregar al modelo una nueva variable asociada a esta columna y con coeficiente igual a 1 en la función objetivo.

In [48]:
# incrementar en 1 el indice maximo de variables
K+= 1

# crear un objeto columna
col = Column()

# definir los coeficientes de la columna usando la solucion del modelo del knapsack
vy = h.getAttr('x', y)
for i in I:
    if vy[i] > 0.1:
        col.addTerms(round(vy[i]), r[i])

# agregar al modelo principal una nueva variable asociada a esta columna
x[K] = pm.addVar(name="x[{}]".format(K), vtype=GRB.INTEGER, obj=1, column= col)

Repetimos el procedimiento de generación de columnas mientras el subproblema de costeo tenga una solución óptima con valor mayor a 1. Para desactivar los mensajes a la pantalla durante la solución de los modelos, fijamos a cero el valor del parámetro `OutputFlag` en ambos modelos

In [49]:
# desactivar mensajes de salida del modelo pm
pm.Params.OutputFlag = 0
while True:
    # 1. Resolver la relajacion lineal del problema master
    # Llamamos primero a update para actualizar el modelo master
    pm.update()
    # Construimos la relajacion lineal de m
    relax = pm.relax()
    # Resolvemos la relacion lineal de m
    relax.optimize() 

        
    # 2. Construir y resolver el subproblema de costeo (PRICING)
    # Recuperamos la solucion dual del problema master
    pi = {}
    i = 1
    for c in relax.getConstrs():
        pi[i] = c.Pi
        i+= 1
    # Crear un objeto modelo auxiliar para el problema de la mochila
    h = Model('knapsack')
    # desactivar mensajes de salida del modelo pm
    h.Params.OutputFlag = 0
    # Crear las variables del problema de la mochila
    y = h.addVars(I, name="y", vtype=GRB.INTEGER)
    # Crear la funcion objetivo
    h.setObjective(y.prod(pi, '*'), GRB.MAXIMIZE)
    # Agregar la restriccion 
    h.addConstr(y.prod(w, '*') <= L, 'capacidad')
    # Resolver modelo del subproblema
    h.optimize()
    
    # 3. Si el valor optimo de la funcion objetivo del subproblema no supera 1, terminar la generacion de columnas
    if h.ObjVal < 1.1:
        break
        
    # 4. Agregar la nueva columna al problema
    # incrementar en 1 el indice maximo de variables
    K+= 1

    # crear un objeto columna
    col = Column()

    # definir los coeficientes de la columna usando la solucion del modelo del knapsack
    vy = h.getAttr('x', y)
    for i in I:
        if vy[i] > 0.1:
            col.addTerms(round(vy[i]), r[i])

    # agregar al modelo principal una nueva variable asociada a esta columna
    x[K] = pm.addVar(name="x[{}]".format(K), vtype=GRB.INTEGER, obj=1, column= col) 

# la fase de generacion de columnas ha terminado
print("Se generaron {} patrones de corte...".format(K-m))

Se generaron 26 patrones de corte...


Finalmente, resolvemos el modelo del problema master como un programa entero. Fijamos un límite para el tiempo de cálculo y una tolerancia para la brecha de optimalidad:

In [52]:
# Terminar al alcanzar un Gap del 10%
pm.Params.MIPGap = 0.1

# Terminar luego de 240 segundos
pm.Params.TimeLimit = 240

# Activar nuevamente los mensajes de salida
pm.Params.OutputFlag = 1

# Resolver el problema principal
pm.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 20 rows, 46 columns and 84 nonzeros
Variable types: 0 continuous, 46 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 5e+01]
Presolved: 19 rows, 37 columns, 75 nonzeros

Continuing optimization...


Explored 1 nodes (25 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 248 279 

Optimal solution found (tolerance 1.00e-01)
Best objective 2.480000000000e+02, best bound 2.450000000000e+02, gap 1.2097%


Escribir la solución óptima:

In [68]:
# Recuperar variables cuyo valor sea mayor a cero:
for var in pm.getVars():
    if var.getAttr('X') > 0.1:
        # Recuperar la columna asociada a cada variable
        col = pm.getCol(var)
        # Decodificar el patron correspondiente a col
        L = []
        for c in pm.getConstrs():
            exito = False
            for i in range(col.size()):
                if col.getConstr(i)== c:
                    L.append(int(col.getCoeff(i)))
                    exito = True
            if not exito:
                L.append(0)
        S = ['{:2} '.format(k) for k in L]
        print ('{:2}: [{}]'.format(int(var.getAttr('X')), ''.join(S)))


 1: [20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
 1: [ 0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
10: [ 0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
 4: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0 ]
26: [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0 ]
 1: [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0 ]
 3: [ 2  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0 ]
13: [ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0 ]
 7: [ 0  0  1  2  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0 ]
14: [ 1  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0 ]
46: [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0 ]
 1: [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0 ]
31: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0 ]
22: [ 1  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 ]
24: [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 

## Código completo

Se reproduce a continuación el código completo del modelo anterior. Notar que no es necesario generar la primera columna "por separado" como lo hicimos en este ejemplo, con el propósito de explicar mejor el algoritmo.

In [2]:
# Implementación de modelos lineales enteros
# Problema del corte de material (Cutting-Stock)
# Implementación con generación de columnas

# Luis M. Torres (EPN 2019)

from gurobipy import *
import random as rm

# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de ordenes
m = 50

# Conjunto de indices para las ordenes, I:={1,...,m}
I = range(1, m+1)

# Diccionarios con las longitudes y cantidades demandadas de cada item
w = {} # longitudes 1, 3, ... , 2m+1
b = {} # demandas (aleatorio entre 1 y 50)
for i in I:
    w[i]= rm.randint(m, 4*m)
    b[i]= rm.randint(10, 20)

L = 5*m

# Este valor se utiliza para indicar el mayor indice de una variable en el modelo
K = m

print w
print b
print L

try:
    # Crear el objeto modelo
    pm = Model('cutting-stock')

    # Desactivar los mensajes del modelo
    pm.Params.OutputFlag = 0

    # Crear las variables asociadas a patrones homogeneos
    x = pm.addVars(I, name="x", vtype=GRB.INTEGER)

    # Crear la funcion objetivo
    pm.setObjective(x.sum('*'), GRB.MINIMIZE)

    # Restricciones de demanda
    r = pm.addConstrs((int(L/w[i])*x[i] >= b[i] for i in I), "demanda");

    # Escribir el modelo PM inicial a un archivo
    # m.write('cutting-stock-inicial.lp')
    
    # lazo principal de la generacion de columna
    while True:
        # 1. Resolver la relajacion lineal del problema master reducido
        # Llamamos primero a update para actualizar el modelo master
        pm.update()
        # Construimos la relajacion lineal de m
        relax = pm.relax()
        # Resolvemos la relacion lineal de m
        relax.optimize() 
        
        # 2. Construir y resolver el subproblema de costeo (PRICING)
        # Recuperamos la solucion dual del problema master
        pi = {}
        i = 1
        for c in relax.getConstrs():
            pi[i] = c.Pi
            i+= 1
        # Crear un objeto modelo auxiliar para el problema de la mochila
        h = Model('knapsack')
        # desactivar mensajes de salida del modelo pm
        h.Params.OutputFlag = 0
        # Crear las variables del problema de la mochila
        y = h.addVars(I, name="y", vtype=GRB.INTEGER)
        # Crear la funcion objetivo
        h.setObjective(y.prod(pi, '*'), GRB.MAXIMIZE)
        # Agregar la restriccion 
        h.addConstr(y.prod(w, '*') <= L, 'capacidad')
        # Resolver modelo del subproblema
        h.optimize()
    
        # 3. Si el valor optimo de la funcion objetivo del subproblema no supera 1, terminar la generacion de columnas
        if h.ObjVal < 1.1:
            break
        
        # 4. Agregar la nueva columna al problema
        # incrementar en 1 el indice maximo de variables
        K+= 1
        # crear un objeto columna
        col = Column()
        # definir los coeficientes de la columna usando la solucion del modelo del knapsack
        vy = h.getAttr('x', y)
        for i in I:
            if vy[i] > 0.1:
                col.addTerms(round(vy[i]), r[i])
        # agregar al modelo principal una nueva variable asociada a esta columna
        x[K] = pm.addVar(name="x[{}]".format(K), vtype=GRB.INTEGER, obj=1, column= col) 
        
    # la fase de generacion de columnas ha terminado
    print("Se generaron {} patrones de corte...".format(K-m))

    # Fijar parametros del modelo principal
    # Terminar al alcanzar un Gap del 10%
    pm.Params.MIPGap = 0.1
    # Terminar luego de 180 segundos
    pm.Params.TimeLimit = 180
    # Activar nuevamente los mensajes de salida
    pm.Params.OutputFlag = 1

    # Resolver el problema principal
    pm.optimize()
    
    # Escribir la solucion
    # Recuperar variables cuyo valor sea mayor a cero:
    for var in pm.getVars():
        if var.getAttr('X') > 0.1:
            # Recuperar la columna asociada a cada variable
            col = pm.getCol(var)
            # Decodificar el patron correspondiente a col
            L = []
            for c in pm.getConstrs():
                exito = False
                for i in range(col.size()):
                    if col.getConstr(i)== c:
                        L.append(int(col.getCoeff(i)))
                        exito = True
                if not exito:
                    L.append(0)
            S = ['{:2} '.format(k) for k in L]
            print ('{:2}: [{}]'.format(int(var.getAttr('X')), ''.join(S)))
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

{1: 177, 2: 113, 3: 127, 4: 168, 5: 121, 6: 187, 7: 92, 8: 143, 9: 187, 10: 172, 11: 96, 12: 185, 13: 121, 14: 115, 15: 187, 16: 122, 17: 89, 18: 132, 19: 158, 20: 174, 21: 50, 22: 181, 23: 99, 24: 78, 25: 86, 26: 171, 27: 62, 28: 126, 29: 66, 30: 156, 31: 172, 32: 195, 33: 138, 34: 140, 35: 136, 36: 78, 37: 142, 38: 121, 39: 164, 40: 189, 41: 185, 42: 131, 43: 156, 44: 172, 45: 185, 46: 193, 47: 118, 48: 200, 49: 169, 50: 142}
{1: 18, 2: 12, 3: 14, 4: 13, 5: 16, 6: 15, 7: 18, 8: 12, 9: 20, 10: 19, 11: 18, 12: 17, 13: 11, 14: 16, 15: 20, 16: 19, 17: 18, 18: 10, 19: 14, 20: 17, 21: 15, 22: 12, 23: 19, 24: 16, 25: 20, 26: 14, 27: 13, 28: 20, 29: 16, 30: 16, 31: 15, 32: 16, 33: 14, 34: 14, 35: 13, 36: 12, 37: 17, 38: 10, 39: 19, 40: 19, 41: 20, 42: 14, 43: 13, 44: 19, 45: 16, 46: 16, 47: 17, 48: 20, 49: 10, 50: 15}
250
Academic license - for non-commercial use only
Se generaron 54 patrones de corte...
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimi